In [2]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from __future__ import print_function

import os
import os.path
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
import json
from pprint import pprint
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from numpy.random import RandomState
prng = RandomState(1234567890)
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors

In [ ]:
with open('C:\\Users\mites\Documents\Cognitive\Midterm\Rawdata\ssix-project-semeval-2017-task-5-subtask-2-2fb645e839b7\Headline_Trainingdata.json') as data_file:    
    data = json.load(data_file)

pprint(data[:5])

In [ ]:
with open('C:\\Users\mites\Documents\Cognitive\Midterm\Rawdata\ssix-project-semeval-2017-task-5-subtask-2-2fb645e839b7\Headlines_Testdata.json',encoding="utf8") as data_files:    
    datatest = json.load(data_files)

pprint(datatest[:5])

In [ ]:
ids = [x["id"] for x in data]
company = [x["company"] for x in data]
title = [x["title"] for x in data]
score = [x["sentiment"] for x in data]

In [ ]:
result= list(map(float, score))
sentscr=[]
for item in result:
   
    if item>=-1 and item<=-0.3:
        sentscr.append(-1)
    elif item>-0.3 and item<=0.3:
        sentscr.append(0)
    else: 
        sentscr.append(1)
        
sentscr[:10]

In [ ]:
df = pd.DataFrame(
    {'id':ids,
     'company': company,
     'sentimentscore': sentscr,
    })
df[:5]

In [ ]:
df1 = pd.DataFrame(
    {'title':title
    })
df1[:5]

In [ ]:
a = df1.title.astype(str).str.replace('\[|\]|\'', '')
a[:5]

In [ ]:
df2=a.to_frame()
df2[:5]

In [ ]:
dataframe = df.join(df2)
dataframe[:10]

In [ ]:
ids = [x["id"] for x in datatest]
company = [x["company"] for x in datatest]
title = [x["title"] for x in datatest]

In [ ]:
dft = pd.DataFrame(
    {'id':ids,
     'company': company,
    })
dft[:10]

In [ ]:
dft1 = pd.DataFrame(
    {'title':title
    })
dft1[:10]

In [ ]:
b = dft1.title.astype(str).str.replace('\[|\]|\'', '')
b[:10]

In [ ]:
dft2=b.to_frame()
dft2[:10]

In [ ]:
testdataframe = dft.join(dft2)
testdataframe[:10]

In [ ]:
testdataframe = testdataframe[['id','company','title']]
testdataframe[:5]

In [ ]:
BASE_DIR = 'C:\\Users\mites\Documents\Cognitive\Midterm'
GLOVE_DIR = BASE_DIR + '\GoogleNews-vectors-negative300'

MAX_SEQUENCE_LENGTH = 3000
MAX_NB_WORDS = 20000

# Consider changing the 200 to 25
EMBEDDING_DIM = 300
GLOVE_FILE = 'C:\\Users\mites\Documents\Cognitive\Midterm\GoogleNews-vectors-negative300\GoogleNews-vectors-negative300.bin'

In [ ]:
print('Indexing word vectors.')

embeddings_index = {} # word -> coefs
# We'll use the 100-dimensional version
#f = open(os.path.join(GLOVE_DIR, GLOVE_FILE), encoding="utf8")
word2vec = KeyedVectors.load_word2vec_format(GLOVE_FILE, binary=True)

In [ ]:
#Selecting the columns except the last one i.e sentiment score
X = dataframe.iloc[:,3].values
Xt = testdataframe.iloc[:,3].values

In [ ]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
data = pad_sequences(sequences, maxlen=3000)

In [ ]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(Xt)
sequences = tokenizer.texts_to_sequences(Xt)
testdata = pad_sequences(sequences, maxlen=3000)

In [ ]:
print(tokenizer.word_index)

In [ ]:
word2index = tokenizer.word_index
len(word2index)

In [ ]:
Y = np.array(dataframe.iloc[:,2].values)
Y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data,Y, test_size=0.15)

In [ ]:
seed = 123
np.random.seed(seed)
np.random.shuffle(x_train)
np.random.seed(seed) # same shuffle for labels!
np.random.shuffle(y_train)

np.random.seed(seed * 2)
np.random.shuffle(x_test)
np.random.seed(seed * 2)
np.random.shuffle(y_test)

In [ ]:
print('Pad sequences')
x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
x_train[x_train >= MAX_NB_WORDS] = MAX_NB_WORDS-1
x_test[x_test >= MAX_NB_WORDS] = MAX_NB_WORDS-1

In [ ]:
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word2index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word2index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
# train a regular MLP
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='float32')
embedded_sequences = embedding_layer(sequence_input)
x = Flatten()(embedded_sequences)
x = Dense(25, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(25, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

model_mlp = Model(sequence_input, preds)
model_mlp.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
model_mlp.summary()

In [ ]:
print('Training model.')
history = model_mlp.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=200, batch_size=32)

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
preds = model_mlp.predict(testdata, batch_size=32, verbose=1)

In [ ]:
abc = preds.astype(int).flatten().flatten()
zxy = pd.DataFrame({'predictedvalues': abc})

In [ ]:
zxy[:5]